# **CHAPTER 9 <br> Decorators and Closures**

# Decorators 101

* A decorator is a callable that takes another function as an argument (the decorated
function).
* A decorator may perform some processing with the decorated function, and returns
it or replaces it with another function or callable object.
* Strictly speaking, decorators are just syntactic sugar.
* You can always simply call a decorator like any regular callable, passing another function. Sometimes that is actually convenient, especially when doing metaprogramming—changing program
behavior at runtime.

**Three essential facts make a good summary of decorators:**
- A decorator is a function or another callable.
- A decorator may replace the decorated function with a different one.
- Decorators are executed immediately when a module is loaded.

In [1]:
# Example 9-1. A decorator usually replaces a function with a different one
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco # target = deco(target)
def target():
    print('running target()')

In [2]:
# Invoking the decorated target actually runs inner.
target()

running inner()


In [3]:
# Inspection reveals that target is a now a reference to inner.
target

<function __main__.deco.<locals>.inner()>

# When Python Executes Decorators

* A key feature of decorators is that they run right after the decorated function is
defined. That is usually at import time (i.e., when a module is loaded by Python).This highlights the difference between what Pythonistas call **import time** and **runtime**.

In [4]:
#Example 9-2. The registration.py module

registry = [] # registry will hold references to functions decorated by @register.

def register(func): # register takes a function as an argument.
    print(f'running register({func})')
    registry.append(func)
    return func # we must return a function; here we return the same received as argument.

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x000001C4FE5E55A0>)
running register(<function f2 at 0x000001C4FE5E57E0>)
running main()
registry -> [<function f1 at 0x000001C4FE5E55A0>, <function f2 at 0x000001C4FE5E57E0>]
running f1()
running f2()
running f3()


Note that register runs (twice) before any other function in the module.

The **main point of Example 9-2** is to emphasize that function decorators are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked.

# Registration Decorators

- Example 9-2 is unusual in two ways:
    - The decorator function is defined in the same module as the decorated functions.A real decorator is usually defined in one module and applied to functions in other modules.
    - The register decorator returns the same function passed as an argument. In practice, most decorators define an inner function and return it to replace the decorated function.

- Even though the register decorator in Example 9-2 returns the decorated function unchanged, that technique is not useless. Similar decorators are used in many Python frameworks **to add functions to some central registry**—for example, a registry mapping
URL patterns to functions that generate HTTP responses. **Such registration decorators may or may not change the decorated function.** (registration decorator example in chapter 10)

- Most decorators do change the decorated function. They usually do it by defining an inner function and returning it to replace the decorated function. Code that uses inner    functions almost always depends on closures to operate correctly.

# Variable Scope Rules

In [5]:
#Example 9-3. Function reading a local and a global variable
def f1(a):

    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [6]:
#Example 9-4. Variable b is local, because it is assigned a value in the body of the function
# This is not a bug, but a design choice

b = 6

def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [7]:
# If we want the interpreter to treat b as a global variable
# and still assign a new value to it within the function

b = 6
def f3(a):
    global b # use the global declaration
    print(a)
    print(b)
    b = 9
f3(3)

3
6


In [8]:
b

9

In the preceding examples, we can see two scopes in action:
- The module global scope
    - Made of names assigned to values outside of any class or function block.
- The f3 function local scope
    - Made of names assigned to values as parameters, or directly in the body of the
function.

In [9]:
#Example 9-5. Disassembly of the f1 function from Example 9-3

from dis import dis

dis(f1)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [10]:
#Example 9-6. Disassembly of the f2 function from Example 9-4

dis(f2)

  7           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  8           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  9          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


### why error happen in Example 9-4 :
- LOAD_GLOBAL (Example 9-5)
- LOAD_FAST (Example 9-6) :
Load local name b. This shows that the compiler considers b a local variable, even
if the assignment to b occurs later, because the nature of the variable—whether it
is local or not—cannot change in the body of the function.

$\color{orange}{Note}$: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local.

# Closures

- A **closure** is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

- Closures are sometimes confused with **anonymous functions**.

- Closures only matter when you have **nested functions**.

- It does not matter whether the function is anonymous or not; what matters is that it can **access nonglobal variables that are defined outside of its body**.

- The **only situation** in which a function may need to deal with **external variables that are nonglobal** is when it is nested in another function and those variables are part of the local scope of the outer function.




In [11]:
#Example 9-7. average_oo.py: a class to calculate a running average

class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

# The Averager class creates instances that are callable   
avg = Averager() 

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [12]:
#Example 9-8. a higher-order function to calculate a running average
#functional implementation of Example 9-7

def make_averager():
    
    series = [] # "series" is a local variable of make_averager 
    
    def averager(new_value):
        series.append(new_value) # within averager "series" is a free variable
        total = sum(series)

        return total / len(series)
    return averager # make_averager() returns an averager function object

# Example 9-9

avg = make_averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


$\color{orange}{Note:}$
- Note the similarities of the last two examples: 
    - we call `Averager()` or `make_averager()` to get a callable object avg .
    - In Example 9-7, avg is an instance of `Averager`.
    - in Example 9-8, it is the inner function, `averager`.
    
- variable `series` is a _local variable_ of `make_averager` because the assignment series
= [ ] happens in the body of that function.
- Within `averager`, `series` is a _free variable_. This is a technical term meaning a variable
that is not bound in the local scope.

- Python keeps the names of local and free variables in the `__code__` attribute that represents the compiled body of the function.(Example 9-10)

- where does the `avg` keeps history in each example?
    - in Example 9-7 It’s obvious the `avg` of the Averager class keeps the history in the `self.series` instance attribute.

    - in Example 9-8 Within averager, `series` is a free variable. The value for `series` is kept in the `__closure__` attribute of the returned function `avg`. Each item in `avg.__closure__` corresponds to a name in `avg. __code__.co_freevars`.<br> These items are cells, and they have an attribute called `cell_contents` where the actual value can be found(Example 9-11).



In [13]:
#Example 9-10. Inspecting the function created by make_averager in Example 9-8
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
('series',)


In [14]:
#Example 9-11

print (avg.__closure__)
print(avg.__closure__[0].cell_contents)

(<cell at 0x000001C4FE896920: list object at 0x000001C4FFABA400>,)
[10, 11, 12]


# The nonlocal Declaration
- previous implementation of make_averager (Example 9-8) was not efficient.

- A better implementation would only store the total and the number of items so far, and compute the mean from these two numbers.

In [15]:
# Example 9-12
#A broken higher-order function to calculate a running average without keeping all history
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager
avg = make_averager()
avg(10) 
#error cause we are actually assigning to count in the body of averager, and that makes it a local variable.

UnboundLocalError: local variable 'count' referenced before assignment

- when count is a number or any **immutable** type, we are actually assigning to count in the body of `averager`, and that makes it a **local variable**.The same problem affects the total variable.

- in Example 9-8 we took advantage of the fact that lists are **mutable**.we never assigned to the `series` name; we only called `series.append` and invoked `sum` and `len` on it.

- with **immutable** types like numbers, strings, tuples, etc., all you can do is **read**,
never update. If you try to rebind them, as in count = count + 1, then you are
implicitly creating a *local variable* count. It is no longer a *free variable*, and therefore
**it is not saved in the closure**.

__*Solution*__, the `nonlocal` keyword:
- nonlocal was introduced in Python 3. 
- It lets you declare a variable as a free variable even when it is assigned within the function. 
- If a new value is assigned to a nonlocal variable, the binding stored in the closure is
changed.

In [16]:
# Example 9-13. example 9-12 fixed with the use of nonlocal
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [17]:
avg = make_averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [18]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value',)
('count', 'total')


In [19]:
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)
print(avg.__closure__[1].cell_contents)


(<cell at 0x000001C4FF962830: int object at 0x000001C4F99A0130>, <cell at 0x000001C4FF91A1A0: int object at 0x000001C4F99A04F0>)
3
33


## Variable Lookup Logic

When a function is defined, the Python bytecode compiler determines how to fetch a
variable x that appears in it, based on these rules:
* If there is a **global** x declaration, x comes from and is assigned to the x global variable module.

* If there is a **nonlocal** x declaration, x comes from and is assigned to the x local variable of the *nearest surrounding function* where x is defined.

* If x is a parameter or is assigned a value in the *function body*, then x is the **local** variable.

* If x is referenced but is *not assigned* and is *not a parameter*:
    * x will be looked up in the *__local__ scopes* of the surrounding function bodies (*__nonlocal__ scopes*).
    * If not found in surrounding scopes, it will be read from the module *__global__ scope*.
    * If not found in the global scope, it will be read from `__builtins__.__dict__`.


# Implementing a Simple Decorator

In [20]:
# Example 9-14. clockdeco0.py: simple decorator to show the running time of functions
import time

def clock(func):
    def clocked(*args): # Define inner function clocked to accept any number of positional arguments.
        t0 = time.perf_counter() 
        result = func(*args) # the closure for the 'clocked' function,encompasses the 'func' free variable.
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked #Return the inner function to replace the decorated function.

In [21]:
# Example 9-15. Using the clock decorator
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock # same as writing  --> factorial = clock(factorial) 
def factorial(n): 
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12375540s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000040s] factorial(1) -> 1
[0.00000650s] factorial(2) -> 2
[0.00002870s] factorial(3) -> 6
[0.00003260s] factorial(4) -> 24
[0.00005520s] factorial(5) -> 120
[0.00007350s] factorial(6) -> 720
6! = 720


$\color{orange}{Note:}$ 
The clock decorator implemented in Example 9-14 has a few _shortcomings_: 
- it does not support keyword arguments
- it masks the `__name__` and `__doc__` of the decorated function.

**solution:** 
- add `**kwargs` to correctly handle keyword arguments 
- using the `functools.wraps` decorator to copy the relevant attributes from func to clocked

In [22]:
# factorial now actually holds a reference to the clocked function.
factorial.__name__

'clocked'

In [23]:
# Example 9-16. clockdeco.py: an improved clock decorator

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked


# Decorators in the Standard Library

- Python has three built-in functions that are designed to **decorate methods**:
    - `property`
    - `classmethod` 
    - `staticmethod`


- we saw another important decorator: `functools.wraps`, a helper for building well-behaved decorators.(Example 9-16)

- Some of the most interesting decorators in the standard library (all from the functools module):
    - `cache`
    - `lru_cache` 
    - `singledispatch`

## Memoization with functools.cache
The `functools.cache` decorator implements **memoization**: an optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments.

Besides making silly recursive algorithms viable, `@cache` really shines in applications
that need to **fetch information from remote APIs**.

- Python 3.9                        &rarr;  `@cache`
- Python 3.8                        &rarr; replace `@cache` with `@lru_cache`   <br>
- prior versions(since Python 3.2 ) &rarr; ` @lru_cache()` 


In [24]:
# Example 9-17
#The very costly recursive way to compute the nth number in the Fibonacci series
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000050s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00008170s] fibonacci(2) -> 1
[0.00000030s] fibonacci(1) -> 1
[0.00000040s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00000940s] fibonacci(2) -> 1
[0.00003210s] fibonacci(3) -> 2
[0.00012790s] fibonacci(4) -> 3
[0.00000020s] fibonacci(1) -> 1
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00000780s] fibonacci(2) -> 1
[0.00001560s] fibonacci(3) -> 2
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00000790s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000030s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00000810s] fibonacci(2) -> 1
[0.00001590s] fibonacci(3) -> 2
[0.00003120s] fibonacci(4) -> 3
[0.00005390s] fibonacci(5) -> 5
[0.00018990s] fibonacci(6) -> 8
8


In [25]:
# Example 9-18
#Faster implementation using caching way  to compute the nth number in the Fibonacci series

@functools.cache
@clock  #stacked decorators: @functools.cache is applied on the function returned by @clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000040s] fibonacci(0) -> 0
[0.00000110s] fibonacci(1) -> 1
[0.00016250s] fibonacci(2) -> 1
[0.00000110s] fibonacci(3) -> 2
[0.00019020s] fibonacci(4) -> 3
[0.00000050s] fibonacci(5) -> 5
[0.00020170s] fibonacci(6) -> 8
8


 ### **Stacked Decorators**
If there’s more than one decorator, they behave like nested function calls. This:<br>

@alpha<br>
@beta<br>
def my_fn():<br>
...<br>

is the same as this:<br>

my_fn = alpha(beta(my_fn))<br>

In other words, **the beta decorator is applied first, and the function it returns is then passed to alpha.**

$\color{orange}{Note:}$  <br>**All the arguments taken by the decorated function must be hashable** <br>
because the underlying `lru_cache` uses a `dict` to store the results, and the keys are made from the positional and keyword arguments used in the calls.

$\color{red}{Warnning:}$ <br>
**`functools.cache` can consume all available memory if there is a
very large number of cache entries.**
- `functools.cache` is more suitable for use in short-lived command-line scripts. 
- In long-running processes, using `functools.lru_cache` with a suitable
maxsize parameter is recommended.

## Using lru_cache
- `lru_cache` function is more flexible and compatible with Python 3.8 and earlier versions.

- The main advantage of @lru_cache is that its memory usage is bounded by the
`maxsize` parameter

- `LRU` stands for Least Recently Used, meaning that older entries that
have not been read for a while are discarded to make room for new ones.
- Since Python 3.8  &rarr; `@lru_cache` or ` @lru_cache()`

- Since Python 3.2  &rarr; `@lru_cache()` 

- `maxsize = 128` : default value of maxsize
- For **optimal performance**, maxsize should be a **power of 2**

- `maxsize = None` &rarr; LRU logic is disabled (cache works faster but entries are never discarded,may consume too much memory like `@cache`)
- `typed = False` Determines whether the results of different argument types are stored separately: <br>
for example float and integer arguments that are considered
equal are stored only once <br> 
`typed = False` &rarr; f(1) == f(1.0) 

## Single Dispatch Generic Functions

The `functools.singledispatch` decorator allows different modules to contribute to
the overall solution, and lets you easily provide **specialized functions** even for types
that belong to **third-party packages** that you can’t edit. <br>

- If you decorate a plain function with `@singledispatch`, it becomes the entry point for a **generic function**: a group of functions to perform the same operation in different ways depending on the type of the *first argument*.

- Note that the dispatch happens on the type of the first argument, this is what is meant by the term **single dispatch**. If more arguments were used to select the specific functions, we’d have **multiple dispatch**.

- A notable quality of the singledispatch mechanism is that you can register specialized functions anywhere in the system, in any module. If you later add a module with a new user-defined type, you can easily provide a new custom function to handle that type. And you can write custom functions for classes that you did not write and can’t change.

- `functools.singledispatch` exists since Python 3.4, but it only supports type hints since Python 3.7.

- `@singledispatch` is not designed to bring Java-style method overloading to Python.

- The advantage of `@singledispatch` is supporting modular extension: each module can register a specialized function for each type it supports.

In [26]:
# Example 9-20
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch # marks the base function that handles the object type.
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'


@htmlize.register # Each specialized function is decorated with @«base».register.
def _(text: str) -> str: # The name of the specialized functions is irrelevant so _ is a good choice to make this clear.
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:  
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register # The numbers ABCs are useful for use with singledispatch
def _(n: numbers.Integral) -> str: 
    return f'<pre>{n} (0x{n:x})</pre>'


# bool is a subtype-of numbers.Integral, but the singledispatch logic seeks the implementation with 
# the most specific matching type, regardless of the order they appear in the code.
@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

# you can pass a type to the @«base».register decorator. This syntax works in Python 3.4 or later.
@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

# The @«base».register decorator returns the undecorated function,
#  so it’s possible to stack them to register two or more types on the same implementation.
@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

$\color{orange}{Note:}$  <br>
- When possible, register the specialized functions to handle ABCs (abstract classes)
such as `numbers.Integral` and `abc.MutableSequence`, instead of concrete implementations
like `int` and `list`.
This allows your code to **support a greater variety of compatible types**.

- Using **ABCs** or **typing.Protocol** with `@singledispatch` allows your code to support existing or future classes that are actual or virtual subclasses of those ABCs, or that implement those protocols (will be discussed in Chapter 13).

In [27]:
# Example 9-19. htmlize() generates HTML tailored to different object types

print('>>> htmlize({1, 2, 3}) \n',htmlize({1, 2, 3}))
# '<pre>{1, 2, 3}</pre>'

print('>>> htmlize(abs) \n', htmlize(abs))
# '<pre>&lt;built-in function abs&gt;</pre>'

print(">>> htmlize('Heimlich & Co.\\n- a game') \n",htmlize('Heimlich & Co.\n- a game'))
# '<p>Heimlich &amp; Co.<br/>\n- a game</p>'

print('>>> htmlize(42) \n',htmlize(42))
# '<pre>42 (0x2a)</pre>'

print(">>> print(htmlize(['alpha', 66, {3, 2, 1}])) \n", htmlize(['alpha', 66, {3, 2, 1}]))

print(">>> htmlize(True)\n",htmlize(True))
# '<pre>True</pre>'

print(">>> htmlize(fractions.Fraction(2, 3)) \n",htmlize(fractions.Fraction(2, 3)))
# '<pre>2/3</pre>'

print(">>> htmlize(2/3) \n",htmlize(2/3))
# '<pre>0.6666666666666666 (2/3)</pre>'

print(">>> htmlize(decimal.Decimal('0.02380952')) \n", htmlize(decimal.Decimal('0.02380952')))
# '<pre>0.02380952 (1/42)</pre>'

>>> htmlize({1, 2, 3}) 
 <pre>{1, 2, 3}</pre>
>>> htmlize(abs) 
 <pre>&lt;built-in function abs&gt;</pre>
>>> htmlize('Heimlich & Co.\n- a game') 
 <p>Heimlich &amp; Co.<br/>
- a game</p>
>>> htmlize(42) 
 <pre>42 (0x2a)</pre>
>>> print(htmlize(['alpha', 66, {3, 2, 1}])) 
 <ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>
>>> htmlize(True)
 <pre>True</pre>
>>> htmlize(fractions.Fraction(2, 3)) 
 <pre>2/3</pre>
>>> htmlize(2/3) 
 <pre>0.6666666666666666 (2/3)</pre>
>>> htmlize(decimal.Decimal('0.02380952')) 
 <pre>0.02380952 (1/42)</pre>


# Parameterized Decorators


To make a parameterized decorator, make a **decorator factory** that takes
arguments other than the decorated function and returns a decorator, which is then applied to the function to be
decorated.

Parameterized decorators almost always involve **at least two nested functions**, maybe
more if you want to use `@functools.wraps` to produce a decorator that provides better
support for more advanced techniques.

In [28]:
# Example 9-21. Abridged registration.py module from Example 9-2, repeated here for convenience
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x000001C4FE5E51B0>)
running main()
registry -> [<function f1 at 0x000001C4FE5E51B0>]
running f1()


### A Parameterized Registration Decorator
The new register function is not a decorator but **a decorator factory**. When called, it returns the actual
decorator that will be applied to the target function.

In [29]:
# Example 9-22. To accept parameters, the new register decorator must be called as a function

registry = set() # registry is now a set, so adding and removing functions is faster

def register(active=True): # register takes an optional keyword argument.
    def decorate(func): # The decorate inner function is the actual decorator
        print('running register'
            f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)

        return func # Because decorate is a decorator, it must return a function.
    return decorate # register is our decorator factory, so it returns decorate.

@register(active=False)
def f1():
    print('running f1()')

@register() # If no parameters are passed, register must still be called as a function, to return the actual decorator
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x000001C4FFAC3880>)
running register(active=True)->decorate(<function f2 at 0x000001C4FE5E55A0>)


In [30]:
# Example 9-23 

# When the module is imported, f2 is in the registry.
print(registry)
print('*' * 40)

# The register() expression returns decorate, which is then applied to f3.
print(register()(f3))
print(registry)
print('*' * 40)

# This call removes f2 from the registry
print(register(active=False)(f2))
print(registry)

{<function f2 at 0x000001C4FE5E55A0>}
****************************************
running register(active=True)->decorate(<function f3 at 0x000001C4FE74EB90>)
<function f3 at 0x000001C4FE74EB90>
{<function f3 at 0x000001C4FE74EB90>, <function f2 at 0x000001C4FE5E55A0>}
****************************************
running register(active=False)->decorate(<function f2 at 0x000001C4FE5E55A0>)
<function f2 at 0x000001C4FE5E55A0>
{<function f3 at 0x000001C4FE74EB90>}


### The Parameterized Clock Decorator
For simplicity, Example 9-24 is based on the initial clock implementation from Example 9-14, and not the improved one from Example 9-16 that uses `@functools.wraps`, adding yet another function layer.

In [31]:
# Example 9-24 . Module clockdeco_param.py: the parameterized clock decorator

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): # clock is our parameterized decorator factory
    def decorate(func):# decorate is the actual decorator.
        def clocked(*_args):# clocked wraps the decorated function.
            t0 = time.perf_counter()
            _result = func(*_args) # _result is the actual result of the decorated function.
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) # _args holds the actual arguments of clocked
            result = repr(_result) # result is the str representation of _result, for display.
            print(fmt.format(**locals())) # Using **locals() here allows any local variable of clocked to
                                          # be referenced in the fmt.
            return _result 
        return clocked
    return decorate

if __name__ == '__main__':

    @clock()
    def snooze(seconds):
        time.sleep(seconds)
        
    for i in range(3):
        snooze(.123)

[0.12315160s] snooze(0.123) -> None
[0.12352800s] snooze(0.123) -> None
[0.12327950s] snooze(0.123) -> None


In [32]:
#Example 9-25

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12299720000009984s
snooze: 0.12351620000117691s
snooze: 0.12356290000025183s


In [33]:
#Example 9-26 

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s


### A Class-Based Clock Decorator
For more sophisticated decorators, a class-based implementation using `__call__` may be easier to read and maintain, but to explain the basic idea of this language feature (decorators), functions are easier to understand.


In [34]:
#Example 9-27. Module clockdeco_cls.py: parameterized clock decorator implemented as class

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock: # clock class is our parameterized decorator factory.
    
    def __init__(self, fmt=DEFAULT_FMT): # The argument passed in the clock(my_format) is assigned to the fmt parameter
        self.fmt = fmt
    
    def __call__(self, func):# When invoked, the instance replaces the decorated function with clocked.
        def clocked(*_args): # clocked wraps the decorated function
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

# Lecturers

1. Mahya Asgarian [LinkedIn](https://www.linkedin.com/in/mahya-asgarian-9a7b13249)
2. Atefeh Hosseini [LinkedIn](https://www.linkedin.com/in/atefeh-hosseini-61160b275)


present date : 2023-11-17

# Reviewers

1. Fahimeh Asaadi, review date: 2023-11-16 [LinkedIn](https://www.linkedin.com/in/fahimehasaadi)
2. Hosein Toodehroosta, review date: 2023-11-16, [LinkedIn](https://www.linkedin.com/in/hossein-toodehroosta?utm_source=share&utm_campaign=share_via&utm_content=profile&utm_medium=android_app)
